In [1]:
import os
import io
import numpy

NEWLINE = '\n'
SKIP_FILES = set(['cmds'])

def read_files(path):
    for root, dir_names, file_names in os.walk(path):
        for path in dir_names:
            read_files(os.path.join(root, path))
    for file_name in file_names:
        if file_name not in SKIP_FILES:
            file_path = os.path.join(root, file_name)
        if os.path.isfile(file_path):
            past_header, lines = False, []
            #f = open(file_path)
            f = io.open(file_path, mode="r", encoding="Latin-1")
            for line in f:
                if past_header:
                    lines.append(line)
                elif line == NEWLINE:
                    past_header = True
            f.close()
            yield file_path, NEWLINE.join(lines)#.decode('cp1252', 'ignore')

In [2]:
from pandas import DataFrame

def build_data_frame(path, classification):
    data_frame = DataFrame({'text': [], 'class': []})
    for file_name, text in read_files(path): # ERRO
        data_frame = data_frame.append(
            DataFrame({'text': [text], 'class': [classification]}, index=[file_name]))
    return data_frame

In [8]:
HAM = 0
SPAM = 1

SOURCES = [
    ('data/spam',         SPAM),
    ('data/spam_2',       SPAM),
    ('data/easy_ham',     HAM),
    ('data/hard_ham',     HAM),
    ]

data = DataFrame({'text': [], 'class': []})
for path, classification in SOURCES:
    data = data.append(build_data_frame(path, classification))

# data = data.reindex(numpy.random.permutation(data.index))

text     How are you doing?\n\n\n\nIf you've been like ...
class                                                  1.0
Name: data/spam/spam/0177.d62ac309d8030ef816f7831c3d5d3f7d, dtype: object


In [25]:
from sklearn.utils import shuffle
data = shuffle(data)

In [26]:
import numpy
from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer()
counts = count_vectorizer.fit_transform(numpy.asarray(data['text']))

In [27]:
from sklearn.naive_bayes import MultinomialNB

classifier = MultinomialNB()
targets = numpy.asarray(data['class'])
classifier.fit(counts, targets)

MultinomialNB()

In [28]:
examples = ['Free Viagra call today!', "I'm going to attend the Linux users group tomorrow."]
example_counts = count_vectorizer.transform(examples)
predictions = classifier.predict(example_counts)
predictions # [1, 0]

array([1., 0.])

In [33]:
# from sklearn.pipeline import Pipeline

# pipeline = Pipeline([
#   ('vectorizer',  CountVectorizer()),
#   ('classifier',  MultinomialNB()) ])

# pipeline.fit(numpy.asarray(data['text']), numpy.asarray(data['class']))
# pipeline.predict(examples) # [1, 0]

In [32]:
# from sklearn.model_selection import KFold

# k_fold = KFold(n=len(data), n_folds=6, indices=False)
# scores = []
# for train_indices, test_indices in k_fold:
#   train_text = numpy.asarray(data[train_indices]['text'])
#   train_y    = numpy.asarray(data[train_indices]['class'])

#   test_text  = numpy.asarray(data[test_indices]['text'])
#   test_y     = numpy.asarray(data[test_indices]['class'])

#   pipeline.fit(train_text, train_y)
#   score = pipeline.score(test_text, test_y)
#   scores.append(score)

# score = sum(scores) / len(scores)

In [ ]:
# pipeline = Pipeline([
#   ('count_vectorizer',   CountVectorizer(ngram_range=(1, 2))),
#   ('classifier',         MultinomialNB()) ])

In [ ]:
# pipeline = Pipeline([
#   ('count_vectorizer',   CountVectorizer(ngram_range=(1, 2))),
#   ('tfidf_transformer',  TfidfTransformer()),
#   ('classifier',         MultinomialNB()) ])

In [ ]:
# from sklearn.naive_bayes import BernoulliNB

# pipeline = Pipeline([
#   ('count_vectorizer',   CountVectorizer(ngram_range=(1, 2))),
#   ('classifier',         BernoulliNB(binarize=0.0)) ])